In [1]:
import os
from __future__ import print_function

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
jars = '--jars /root/lib/jpmml-sparkml-package-1.2-SNAPSHOT.jar'
#packages = '--packages org.jpmml:jpmml-sparkml:1.2.0'
packages = ''
py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + jars \
  + ' ' + packages \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

--master local[1] --jars /root/lib/jpmml-sparkml-package-1.2-SNAPSHOT.jar  --py-files /root/lib/jpmml.py pyspark-shell


In [2]:
from py4j.java_gateway import JavaObject
from pyspark.ml import Pipeline

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)

df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df['species'] = pd.factorize(df['species'])[0]
df.columns = ["sepal_length(cm)", "sepal_width(cm)",
                                        "petal_length(cm)",
"petal_width(cm)", "species"]

dfiris = sqlContext.createDataFrame(df)

dfiris.printSchema()

dfiris.show(5)

train, test = dfiris.randomSplit([0.7, 0.3])

train.count()
test.count()

root
 |-- sepal_length(cm): double (nullable = true)
 |-- sepal_width(cm): double (nullable = true)
 |-- petal_length(cm): double (nullable = true)
 |-- petal_width(cm): double (nullable = true)
 |-- species: long (nullable = true)

+----------------+---------------+----------------+---------------+-------+
|sepal_length(cm)|sepal_width(cm)|petal_length(cm)|petal_width(cm)|species|
+----------------+---------------+----------------+---------------+-------+
|             5.1|            3.5|             1.4|            0.2|      0|
|             4.9|            3.0|             1.4|            0.2|      0|
|             4.7|            3.2|             1.3|            0.2|      0|
|             4.6|            3.1|             1.5|            0.2|      0|
|             5.0|            3.6|             1.4|            0.2|      0|
+----------------+---------------+----------------+---------------+-------+
only showing top 5 rows



51

In [4]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=["sepal_length(cm)",
"sepal_width(cm)",
                                        "petal_length(cm)",
"petal_width(cm)"], outputCol="features")

lr = LogisticRegression(featuresCol='features', labelCol='species')

pipeline = Pipeline(stages=[assembler, lr])

lr_model = pipeline.fit(train)

print(lr_model)

PipelineModel_47c58505cc1cd0650d92


In [5]:
prediction_lr = lr_model.transform(test)

score_lr = prediction_lr.select(['species', 'prediction'])
score_lr.show(n=score_lr.count())

prediction_lr.show(5)

+-------+----------+
|species|prediction|
+-------+----------+
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      2|       2.0|
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      0|       0.0|
|      1|       1.0|
|      0|       0.0|
|      1|       1.0|
|      1|       1.0|
|      1|       1.0|
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      2|       2.0|
|      1|       1.0|
|      2|       2.0|
|      1|       2.0|
|      1|       1.0|
|      1|       1.0|
|      2|       1.0|
|      2|       2.0|
|      1|       1.0|
|      2|       2.0|
|      2|       2.0|
|      2|       2.0|
|      1|       1.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      2|       2.0|
|      1|       1.0|
|      1|    

In [6]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sc, train, lr_model)
print(pmmlBytes.decode("UTF-8"))

Py4JJavaError: An error occurred while calling z:org.jpmml.sparkml.ConverterUtil.toPMMLByteArray.
: java.lang.IllegalArgumentException
	at org.jpmml.sparkml.model.LogisticRegressionModelConverter.encodeModel(LogisticRegressionModelConverter.java:55)
	at org.jpmml.sparkml.model.LogisticRegressionModelConverter.encodeModel(LogisticRegressionModelConverter.java:39)
	at org.jpmml.sparkml.ConverterUtil.toPMML(ConverterUtil.java:85)
	at org.jpmml.sparkml.ConverterUtil.toPMMLByteArray(ConverterUtil.java:180)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.Abstract